In [1]:
import os
import math
import wandb
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt


from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *

from tensorflow.keras.regularizers import *

from tensorflow.keras.callbacks import *
from keras.preprocessing.image import *
from tensorflow.keras.preprocessing import *

2023-04-22 01:27:02.530902: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-22 01:27:02.649561: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

# Normalize pixel values between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot encoded vectors
y_train = tf.keras.utils.to_categorical(y_train, num_classes=100)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=100)

In [3]:
# SwiLU 함수 정의 (alpha값 항상 1로 고정)
def serlu(x, lamda = 1.0786, alpha= 1):
    return tf.where(x>=0, x, alpha * x * tf.exp(x))

In [37]:
inputs = Input(shape=(32,32, 3))

# Layer 1
x = Conv2D(32, (3,3), activation=serlu, kernel_regularizer=l2(1e-6))(inputs)

# Layer 2
x = Conv2D(32, (3,3), activation=serlu, kernel_regularizer=l2(1e-6))(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)

# Layer 3
x = Conv2D(64, (3,3), activation=serlu, kernel_regularizer=l2(1e-6))(x)

# Layer 4
x = Conv2D(64, (3,3), activation=serlu, kernel_regularizer=l2(1e-6))(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)

# Layer 5
x = AveragePooling2D()(x)
x = Flatten()(x)
x = Dense(512, activation=serlu, kernel_regularizer=l2(1e-6))(x)

# Layer 6
outputs = Dense(100, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_20 (Conv2D)          (None, 30, 30, 32)        896       
                                                                 
 conv2d_21 (Conv2D)          (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                            
                                                                 
 dropout_10 (Dropout)        (None, 14, 14, 32)        0         
                                                                 
 conv2d_22 (Conv2D)          (None, 12, 12, 64)        18496     
                                                          

In [38]:
# 옵티마이저 및 학습률 스케줄러 정의
# 옵티마이저 정의
learning_rate = 1e-3

def learning_rate_schedule(epoch):
    new_learning_rate = learning_rate

    if epoch <= 50:
        pass
    elif epoch > 50 and epoch <= 100:
        new_learning_rate = learning_rate * 0.1
    else:
        new_learning_rate = learning_rate * 0.01
        
    print('Learning rate:', new_learning_rate)
    
    return new_learning_rate


opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate )


model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [39]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip= True
                             )

# 이미지 데이터의 분포를 조정
datagen.fit(x_train)

In [40]:
from tensorflow.keras import callbacks

wandb.init(project="CIFAR100", entity="hcim", name='LeNet-swilu')

wandbCallback=wandb.keras.WandbCallback(monitor="val_accuracy", mode="max", 
                                        log_weights=True, log_gradients=True, 
                                        training_data=datagen.flow(x_train, y_train, batch_size=128))


callbacks = [wandbCallback, LearningRateScheduler(learning_rate_schedule)]

In [41]:
model.fit(datagen.flow(x_train, y_train, batch_size=128),
          validation_data=(x_test, y_test),
          epochs=200, workers=4,
          callbacks=callbacks)

Learning rate: 0.001
Epoch 1/200
  1/391 [..............................] - ETA: 5:19 - loss: 4.6096 - accuracy: 0.0156

2023-04-22 02:08:23.743985: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape ingradient_tape/model_11/conv2d_23/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


388/391 [============================>.] - ETA: 0s - loss: 3.8789 - accuracy: 0.1089

2023-04-22 02:08:29.224912: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape ingradient_tape/model_12/model_11/conv2d_23/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230422_020815-zl73pz92/files/model-best)... Done. 0.0s


391/391 [==============================] - 14s 33ms/step - loss: 3.8767 - accuracy: 0.1092 - val_loss: 3.4766 - val_accuracy: 0.1733 - lr: 0.0010
Learning rate: 0.001
Epoch 2/200
389/391 [============================>.] - ETA: 0s - loss: 3.2799 - accuracy: 0.2126

...

391/391 [==============================] - 13s 32ms/step - loss: 1.1028 - accuracy: 0.6824 - val_loss: 2.1204 - val_accuracy: 0.4896 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 55/200
388/391 [============================>.] - ETA: 0s - loss: 1.1024 - accuracy: 0.6847

wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230422_020815-zl73pz92/files/model-best)... Done. 0.0s


391/391 [==============================] - 12s 31ms/step - loss: 1.1029 - accuracy: 0.6844 - val_loss: 2.1231 - val_accuracy: 0.4902 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 56/200
391/391 [==============================] - 10s 27ms/step - loss: 1.0888 - accuracy: 0.6871 - val_loss: 2.1264 - val_accuracy: 0.4902 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 57/200
391/391 [==============================] - 10s 26ms/step - loss: 1.0892 - accuracy: 0.6888 - val_loss: 2.1299 - val_accuracy: 0.4900 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 58/200
390/391 [============================>.] - ETA: 0s - loss: 1.0721 - accuracy: 0.6942

wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230422_020815-zl73pz92/files/model-best)... Done. 0.0s


391/391 [==============================] - 12s 30ms/step - loss: 1.0718 - accuracy: 0.6944 - val_loss: 2.1244 - val_accuracy: 0.4946 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 59/200
391/391 [==============================] - 11s 27ms/step - loss: 1.0710 - accuracy: 0.6929 - val_loss: 2.1341 - val_accuracy: 0.4912 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 60/200
391/391 [==============================] - 11s 27ms/step - loss: 1.0697 - accuracy: 0.6922 - val_loss: 2.1378 - val_accuracy: 0.4898 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 61/200
391/391 [==============================] - 11s 27ms/step - loss: 1.0564 - accuracy: 0.6982 - val_loss: 2.1354 - val_accuracy: 0.4925 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 62/200
391/391 [==============================] - 11s 28ms/step - loss: 1.0573 - accuracy: 0.6952 - val_loss: 2.1411 - val_accuracy: 0.4923 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 63/200
391/391 [==============================] - 11s 27ms/step - loss: 1.0546 - a